## install osmium-tool for effiecent extraction (macos only, for windows use conda)
[Anaconda.org](https://anaconda.org/conda-forge/osmium-tool)

In [1]:
#!brew install osmium-tool

!conda install conda-forge::osmium-tool

'conda' is not recognized as an internal or external command,
operable program or batch file.


# Extract relevant OSM Features of Germany, convert the cooridate system to EPSG:25832 and convert to GeoJson with *ogr2ogr*

In [1]:
# Extrahiere relevante OSM-Features direkt aus der deutschlandweiten OSM-Datei
import os

OSM_FILE = "data/germany-latest.osm.pbf"

# Definition der zu extrahierenden Features und deren Filter
EXTRACTIONS = [
    # (feature_name, filter, add_km2_density, add_population_density)
    ("bio_shops", 'nwr/shop=greengrocer nwr/shop=health_food nwr/shop=farm', False, False),
    ("playgrounds", 'nwr/leisure=playground', False, False),
    ("hotels_hostels", 'nwr/tourism=hotel nwr/tourism=hostel', False, False),
    ("gastronomy", 'nwr/amenity=restaurant nwr/amenity=fast_food nwr/amenity=cafe nwr/amenity=bar', False, False),
    ("fast_food", 'nwr/amenity=fast_food', False, False),
    ("schools", 'nwr/amenity=school', False, False),
    ("kindergartens", 'nwr/amenity=kindergarten', False, False),
    ("green_areas", 'nwr/leisure=park nwr/natural=wood', False, False),
    ("water_bodies", 'nwr/natural=water', False, False),
    ("landuse", 'nwr/landuse=residential nwr/landuse=commercial nwr/landuse=industrial nwr/landuse=farm', False, False),
    ("building_types", 'nwr/building=residential nwr/building=commercial nwr/building=apartments', False, False),
    ("all_buildings", 'nwr/building', False, False),
    ("buildings_with_height_levels", 'nwr/building building:levels building:height', False, False),
    ("health", 'nwr/amenity=doctors nwr/amenity=pharmacy', False, False),
    ("finance", 'nwr/amenity=bank nwr/amenity=atm', False, False),
    ("all_highways", 'w/highway', False, False),
    ("cycleways", 'w/highway=cycleway', False, False)
]

# Extrahiere .osm.pbf-Dateien
for name, filter_str, _, _ in EXTRACTIONS:
    pbf_path = f"data/{name}.osm.pbf"
    if not os.path.exists(pbf_path):
        !osmium tags-filter "$OSM_FILE" {filter_str} -o {pbf_path} --overwrite
        print(f"Extrahiert: {pbf_path}")
    else:
        print(f"Bereits vorhanden: {pbf_path}")

# Konvertiere alle extrahierten .osm.pbf-Dateien zu GeoJSON
PBF_GEOJSON_PAIRS = [
    (f"data/{name}.osm.pbf", f"data/{name}.geojson") for name, _, _, _ in EXTRACTIONS
]
# Prüfe, ob GeoJSON-Dateien existieren und konvertiere sie bei Bedarf
for pbf, geojson in PBF_GEOJSON_PAIRS:
    if not os.path.exists(geojson) and os.path.exists(pbf):
        !ogr2ogr -f GeoJSON {geojson} {pbf} -overwrite -sql "SELECT * FROM points" -t_srs EPSG:25832
        print(f"Konvertiert: {pbf} → {geojson}")
    elif not os.path.exists(pbf):
        print(f"Fehlt: {pbf}")
    else:
        print(f"Bereits vorhanden: {geojson}")


Extrahiert: data/bio_shops.osm.pbf


'osmium' is not recognized as an internal or external command,
operable program or batch file.


Extrahiert: data/playgrounds.osm.pbf


'osmium' is not recognized as an internal or external command,
operable program or batch file.


Extrahiert: data/hotels_hostels.osm.pbf


...

Extrahiert: data/all_highways.osm.pbf


'osmium' is not recognized as an internal or external command,
operable program or batch file.


Extrahiert: data/cycleways.osm.pbf


'osmium' is not recognized as an internal or external command,
operable program or batch file.


Fehlt: data/bio_shops.osm.pbf
Fehlt: data/playgrounds.osm.pbf
Fehlt: data/hotels_hostels.osm.pbf
Fehlt: data/gastronomy.osm.pbf
Fehlt: data/fast_food.osm.pbf
Fehlt: data/schools.osm.pbf
Fehlt: data/kindergartens.osm.pbf
Fehlt: data/green_areas.osm.pbf
Fehlt: data/water_bodies.osm.pbf
Fehlt: data/landuse.osm.pbf
Fehlt: data/building_types.osm.pbf
Fehlt: data/all_buildings.osm.pbf
Fehlt: data/buildings_with_height_levels.osm.pbf
Fehlt: data/health.osm.pbf
Fehlt: data/finance.osm.pbf
Fehlt: data/all_highways.osm.pbf
Fehlt: data/cycleways.osm.pbf


## Imports

In [2]:
import geopandas as gpd
import pandas as pd
import os

## Konfiguration

In [3]:
GEMEINDEN_GPKG = "data/DE_VG5000.gpkg"
LAYER = "v_vg5000_gem"
GEMEINDE_ID_COLUMN = "Gemeindeschlüssel_AGS"

DEUTSCHLAND_ATALAS = "data/DeutschlandAtlasData.csv"

EXTRA_FEATURES = "data/bio_shops.geojson"
EXTRA_FEATURES_NAME = "Bio-Läden"


ADD_KM2_DENSITY_COLUMN = False
ADD_POPULATION_DENSITY_COLUMN = False

# --- Überprüfung der Dateipfade ---
if not os.path.exists(GEMEINDEN_GPKG):
    print(f"Fehler: Das GeoPackage der Gemeinden wurde nicht gefunden unter: {GEMEINDEN_GPKG}")
    print("Bitte passen Sie den Pfad 'GEMEINDEN_GPKG' im Skript an.")
    exit()

if not os.path.exists(EXTRA_FEATURES):
    print(f"Fehler: Die GeoJSON-Datei der Bio-Läden wurde nicht gefunden unter: {EXTRA_FEATURES}")
    print("Stellen Sie sicher, dass Sie 'ogr2ogr -f GeoJSON bio_shops.geojson bio_shops.osm.pbf' ausgeführt haben.")
    exit()

Fehler: Die GeoJSON-Datei der Bio-Läden wurde nicht gefunden unter: data/bio_shops.geojson
Stellen Sie sicher, dass Sie 'ogr2ogr -f GeoJSON bio_shops.geojson bio_shops.osm.pbf' ausgeführt haben.


# Adding Deutschland Atalas data

In [ ]:
da_df = pd.read_csv(DEUTSCHLAND_ATALAS, sep=';', decimal=',')
da_df["ID"] = da_df["ID"].astype(str)

gemeinden_gdf = gemeinden_gdf.merge(da_df, left_on=GEMEINDE_ID_COLUMN, right_on="ID", how="left")

NameError: name 'gemeinden_gdf' is not defined

: 

## Verification

In [ ]:

try:
    gemeinden_gdf = gpd.read_file(GEMEINDEN_GPKG, layer=LAYER)
    extras_gdf = gpd.read_file(EXTRA_FEATURES)
    
    print(f"Gemeinden geladen. Anzahl der Gemeinden: {len(gemeinden_gdf)}")
    if GEMEINDE_ID_COLUMN not in gemeinden_gdf.columns:
        print(f"Warnung: Die angegebene Gemeinden-ID-Spalte '{GEMEINDE_ID_COLUMN}' wurde nicht gefunden.")
        print(f"Verfügbare Spalten: {gemeinden_gdf.columns.tolist()}")
        print("Bitte korrigieren Sie 'GEMEINDE_ID_COLUMN' im Skript, um Fortzufahren.")
        exit()

except Exception as e:
    print(f"Fehler beim Laden des Gemeinden GeoPackage: {e}")
    exit()

try:
    print(f"Zusätzliche Features geladen. Anzahl der Features: {len(extras_gdf)}")
    # Sicherstellen, dass das CRS übereinstimmt
    if gemeinden_gdf.crs != extras_gdf.crs:
        print(f"CRS-Konflikt! Gemeinden CRS: {gemeinden_gdf.crs}, Zusätzliche Features CRS: {extras_gdf.crs}")
        exit()

except Exception as e:
    print(f"Fehler beim Laden der GeoJSON: {e}")
    exit()


Gemeinden geladen. Anzahl der Gemeinden: 10957
Zusätzliche Features geladen. Anzahl der Features: 7619


## Geojoin extra featuers with Gemeinden Geopackage

In [ ]:
joined_data = gpd.sjoin(extras_gdf, gemeinden_gdf, how="inner", predicate="within")

print(f"Räumlicher Join abgeschlossen. Anzahl der gejoiniten Einträge: {len(joined_data)}")

counts = joined_data.groupby(GEMEINDE_ID_COLUMN).size().reset_index(name=EXTRA_FEATURES_NAME)

# Fügen Sie die Zählungen wieder dem ursprünglichen Gemeinden-GeoDataFrame hinzu
gemeinden_gdf = gemeinden_gdf.merge(counts, on=GEMEINDE_ID_COLUMN, how="left")
# Ersetzen Sie NaN-Werte (Gemeinden ohne Läden) durch 0
gemeinden_gdf[EXTRA_FEATURES_NAME] = gemeinden_gdf[EXTRA_FEATURES_NAME].fillna(0).astype(int)

if gemeinden_gdf.crs.is_geographic: # Prüft, ob es ein geografisches CRS ist (wie WGS84 - 4326)
    print("Konvertiere Gemeinden-CRS für genaue Flächenberechnung (z.B. nach EPSG:3035)...")
    gemeinden_gdf_proj = gemeinden_gdf.to_crs(epsg=3035) # ETRS89 / LAEA Europe
    gemeinden_gdf['Flaeche_km2'] = gemeinden_gdf_proj.geometry.area / 1_000_000 # Fläche in km²
    print("Flächenberechnung in km² durchgeführt.")
else:
    print("Gemeinden-CRS ist bereits projiziert. Berechne Fläche in km²...")
    gemeinden_gdf['Flaeche_km2'] = gemeinden_gdf.geometry.area / 1_000_000 # Annahme: CRS-Einheit ist Meter
    print("Flächenberechnung in km² durchgeführt.")


Räumlicher Join abgeschlossen. Anzahl der gejoiniten Einträge: 7591
Gemeinden-CRS ist bereits projiziert. Berechne Fläche in km²...
Flächenberechnung in km² durchgeführt.


In [ ]:
# Liste der zu joinenden GeoJSON-Dateien und deren Spaltennamen
def get_feature_files():
    return [
        (f"data/{name}.geojson", name.replace('_', ' ').title().replace(' ', '_'))
        for name, _, _, _ in EXTRACTIONS
    ]

feature_files = get_feature_files()

for file_path, feature_name in feature_files:
    if not os.path.exists(file_path):
        print(f"Warnung: Datei {file_path} nicht gefunden. Überspringe {feature_name}.")
        gemeinden_gdf[feature_name] = 0
        continue
    try:
        extras_gdf = gpd.read_file(file_path)
        if gemeinden_gdf.crs != extras_gdf.crs:
            extras_gdf = extras_gdf.to_crs(gemeinden_gdf.crs)
        joined = gpd.sjoin(extras_gdf, gemeinden_gdf, how="inner", predicate="within")
        counts = joined.groupby(GEMEINDE_ID_COLUMN).size().reset_index(name=feature_name)
        gemeinden_gdf = gemeinden_gdf.merge(counts, on=GEMEINDE_ID_COLUMN, how="left")
        gemeinden_gdf[feature_name] = gemeinden_gdf[feature_name].fillna(0).astype(int)
        print(f"{feature_name}: Join und Zählung abgeschlossen.")
    except Exception as e:
        print(f"Fehler beim Join für {feature_name}: {e}")
        gemeinden_gdf[feature_name] = 0


Bio_Shops: Join und Zählung abgeschlossen.
Playgrounds: Join und Zählung abgeschlossen.
Playgrounds: Join und Zählung abgeschlossen.
Hotels_Hostels: Join und Zählung abgeschlossen.
Warnung: Datei data/gastronomy.geojson nicht gefunden. Überspringe Gastronomy.
Hotels_Hostels: Join und Zählung abgeschlossen.
Warnung: Datei data/gastronomy.geojson nicht gefunden. Überspringe Gastronomy.
Fast_Food: Join und Zählung abgeschlossen.
Schools: Join und Zählung abgeschlossen.
Fast_Food: Join und Zählung abgeschlossen.
Schools: Join und Zählung abgeschlossen.
Kindergartens: Join und Zählung abgeschlossen.
Green_Areas: Join und Zählung abgeschlossen.
Kindergartens: Join und Zählung abgeschlossen.
Green_Areas: Join und Zählung abgeschlossen.
Water_Bodies: Join und Zählung abgeschlossen.
Water_Bodies: Join und Zählung abgeschlossen.
Landuse: Join und Zählung abgeschlossen.
Landuse: Join und Zählung abgeschlossen.
Building_Types: Join und Zählung abgeschlossen.
Building_Types: Join und Zählung abgesc

In [ ]:
# Dichte pro km² hinzufügen, wenn gewünscht
if ADD_KM2_DENSITY_COLUMN:
    NAME_DENSITY_COLUMN = 'Dichte_' + EXTRA_FEATURES_NAME + '_pro_km2'
    gemeinden_gdf[NAME_DENSITY_COLUMN] = (
        gemeinden_gdf[EXTRA_FEATURES_NAME] / gemeinden_gdf['Flaeche_km2']
    ).replace([float('inf'), -float('inf')], 0).fillna(0) # Unendlich durch 0 ersetzen, NaN durch 0


In [ ]:
# Dichte pro Einwohner hinzufügen, wenn gewünscht
if ADD_POPULATION_DENSITY_COLUMN:
    NAME_DENSITY_COLUMN = 'Dichte_' + EXTRA_FEATURES_NAME + '_pro_Einwohner'
    gemeinden_gdf['Einwohner'] = (
        gemeinden_gdf['bev_dicht'] * gemeinden_gdf['Flaeche_km2'])
    gemeinden_gdf[NAME_DENSITY_COLUMN] = (
        gemeinden_gdf[EXTRA_FEATURES_NAME] / gemeinden_gdf['Einwohner']
    ).replace([float('inf'), -float('inf')], 0).fillna(0) # Unendlich durch 0 ersetzen, NaN durch 0


In [ ]:

# Dichte pro km² und pro Einwohner für alle Features, die gewünscht sind
for name, _, add_km2_density, add_population_density in EXTRACTIONS:
    feature_col = name.replace('_', ' ').title().replace(' ', '_')
    if add_km2_density and feature_col in gemeinden_gdf.columns:
        density_col = f'Dichte_{feature_col}_pro_km2'
        gemeinden_gdf[density_col] = (
            gemeinden_gdf[feature_col] / gemeinden_gdf['Flaeche_km2']
        ).replace([float('inf'), -float('inf')], 0).fillna(0)
    if add_population_density and feature_col in gemeinden_gdf.columns:
        density_col = f'Dichte_{feature_col}_pro_Einwohner'
        if 'Einwohner' not in gemeinden_gdf.columns:
            gemeinden_gdf['Einwohner'] = (
                gemeinden_gdf['bev_dicht'] * gemeinden_gdf['Flaeche_km2'])
        gemeinden_gdf[density_col] = (
            gemeinden_gdf[feature_col] / gemeinden_gdf['Einwohner']
        ).replace([float('inf'), -float('inf')], 0).fillna(0)


## Pickle them for later use and shareability

In [ ]:
import pickle

with open("data/gemeinden_gdf.pkl", "wb") as f:
    pickle.dump(gemeinden_gdf, f)